In [2]:
#Hate Speech Classification
#@darknurd

In [3]:
import pandas as pd
import numpy as np
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
train_data = pd.read_csv('train_data.csv', header=None, delimiter="\t", quoting=3)
train_data.columns = ["Sentiment","Text"]
test_data = pd.read_csv('test_data.csv', header=None, delimiter="\t", quoting=3)
test_data.columns = ["Text"]
my_data = pd.read_csv('test1.csv', header=None, delimiter="\t", quoting=3)
my_data.columns = ["Text"]

In [5]:
train_data.shape

(30459, 2)

In [6]:
test_data.shape

(32541, 1)

In [7]:
train_data.head()

,Sentiment,Text
0,0,In the beginning God created the heavens and t...
1,1,"And God said, Let there be light, and there wa..."
2,1,"And God said, Let there be a vault between the..."
3,1,"And God said, Let the water under the sky be g..."
4,1,"Then God said, Let the land produce vegetation..."


In [8]:
test_data.head()

,Text
0,Then we had stupid trivia about San Francisco ...
1,"This means we beat out schools like MIT, which..."
2,i'm off to harvard square bitches..
3,"I'm a big fan of Lakers, so I kind of have all..."
4,seattle sucks!!!...


In [9]:
my_data.head()

,Text
0,you are my fuck


In [10]:
#counting how many labels do we have for each sentiment class.
train_data.Sentiment.value_counts()

1    17719
0    12740
Name: Sentiment, dtype: int64

In [11]:
#Finally, let's calculate the average number of words per sentence. We could do the following using a list comprehension with the number of words per sentence.
np.mean([len(s.split(" ")) for s in train_data.Text])

10.041071604451886

In [12]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # remove non letters
    text = re.sub("[^a-zA-Z]", " ", text)
    # tokenize
    tokens = nltk.word_tokenize(text)
    # stem
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = CountVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = 'english',
                max_features = 85
            )

In [13]:
'''The method fit_transform does two functions: 
    First, it fits the model and learns the vocabulary; 
    second, it transforms our corpus data into feature vectors. 
    The input to fit_transform should be a list of strings, 
    so we concatenate train and test data as follows.'''

'The method fit_transform does two functions: \n    First, it fits the model and learns the vocabulary; \n    second, it transforms our corpus data into feature vectors. \n    The input to fit_transform should be a list of strings, \n    so we concatenate train and test data as follows.'

In [14]:
corpus_data_features = vectorizer.fit_transform(train_data.Text.tolist() + test_data.Text.tolist())
corpus_data_features[0]

<1x85 sparse matrix of type '<type 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [18]:
print (corpus_data_features[3])
#converting to array
corpus_data_features_nd = corpus_data_features.toarray()
corpus_data_features_nd.shape

  (0, 19)	1
  (0, 60)	1
  (0, 80)	2


(63000, 85)

In [26]:
# A look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'aaa', u'airlin', u'amaz', u'angelina', u'awesom', u'beauti', u'becaus', u'boston', u'brokeback', u'citi', u'code', u'cool', u'cruis', u'd', u'da', u'francisco', u'friend', u'fuck', u'geico', u'good', u'got', u'great', u'ha', u'harri', u'harvard', u'hate', u'hi', u'hilton', u'honda', u'imposs', u'joli', u'just', u'know', u'laker', u'left', u'like', u'littl', u'london', u'look', u'love', u'm', u'macbook', u'make', u'miss', u'mission', u'mit', u'mountain', u'movi', u'need', u'new', u'oh', u'onli', u'pari', u'peopl', u'potter', u'purdu', u'realli', u'right', u'rock', u's', u'said', u'san', u'say', u'school', u'seattl', u'shanghai', u'stori', u'stuff', u'stupid', u'suck', u't', u'thi', u'thing', u'think', u'time', u'tom', u'toyota', u'ucla', u've', u'vinci', u'wa', u'want', u'way', u'whi', u'work']


In [16]:
# Sum up the counts of each vocabulary word
dist = np.sum(corpus_data_features_nd, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

1901 aaa
669 airlin
882 amaz
2551 angelina
4845 awesom
3065 beauti
2489 becaus
4050 boston
2000 brokeback
691 citi
2003 code
648 cool
3459 cruis
812 d
2089 da
3215 francisco
669 friend
672 fuck
1934 geico
1191 good
881 got
2085 great
1118 ha
2094 harri
3401 harvard
8023 hate
1214 hi
3671 hilton
3665 honda
1098 imposs
2549 joli
1398 just
1309 know
3775 laker
670 left
5528 like
865 littl
3779 london
1190 look
17806 love
2534 m
1841 macbook
846 make
1488 miss
1102 mission
2267 mit
2167 mountain
1224 movi
1803 need
772 new
902 oh
954 onli
3682 pari
1421 peopl
2093 potter
1787 purdu
2993 realli
855 right
677 rock
5708 s
705 said
3429 san
973 say
682 school
3800 seattl
2031 shanghai
741 stori
682 stuff
5064 stupid
7508 suck
2221 t
2487 thi
903 thing
1830 think
1345 time
3546 tom
3138 toyota
3329 ucla
1051 ve
2001 vinci
5187 wa
2452 want
1365 way
790 whi
652 work


In [43]:
# remember that corpus_data_features_nd contains all of our original train and test data, so we need to exclude
# the unlabeled test entries
X_train, X_test, y_train, y_test  = train_test_split(
    corpus_data_features_nd[0:len(train_data)], 
    train_data.Sentiment,
    train_size=0.80, 
    random_state=42)

In [44]:
#Model Classifier
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)

In [19]:
y_pred = log_model.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.99      0.98      0.99      2512
          1       0.99      0.99      0.99      3580

avg / total       0.99      0.99      0.99      6092



In [34]:
# train classifier
log_model = LogisticRegression()
log_model = log_model.fit(X=corpus_data_features_nd[0:len(train_data)], y=train_data.Sentiment)

# get predictions
test_pred = log_model.predict(corpus_data_features_nd[(len(train_data)):])

print(test_pred.shape)

# sample some of them
import random
spl = random.sample(xrange(len(test_pred)), 5)
print spl
# print text and labels
for text, sentiment in zip(test_data.Text[spl], test_pred[spl]):
    print sentiment, text

(32541,)
[14757, 14213, 27101, 368, 24886]
0 tom cruise sucks.
0 " Angelina Jolie to play evil queen in'Beowulf'"..
0 stupid lakers!!!!!!!!!!!
0 and i blame angelina jolie!
0 actually the only thing that i really like about purdue is my apartment....
